# Attempting to use Prism data as Cloud Optimized GeoTiff

In [ ]:
import rioxarray as rxr
import requests
import zipfile
import io

In [ ]:
date = "20250719" # CANNOT access any one file more than once each day. 

url = f"https://services.nacse.org/prism/data/get/us/800m/ppt/{date}"


In [ ]:
response = requests.get(url)
response.raise_for_status()

In [ ]:
with zipfile.ZipFile(io.BytesIO(response.content)) as z:
    tif_filename = [f for f in z.namelist() if f.endswith(".tif")][0]
    with z.open(tif_filename) as tif_file:
        with open(tif_filename, "wb") as f:
            f.write(tif_file.read())

In [ ]:
ds_prism_chunk = rxr.open_rasterio(tif_filename, masked=True, chunks={"x": 1024, "y": 1024})
%timeit ds_prism_chunk.mean().compute()

In [ ]:
ds_prism = rxr.open_rasterio(tif_filename, masked=True)
%timeit ds_prism.mean()

In [ ]:
%time ds_chunk_5070 = ds_prism_chunk.rio.reproject("EPSG:5070")

In [ ]:
%time ds_5070 = ds_prism.rio.reproject("EPSG:5070")

Dask array with chunks of 1024 x 1024 is slower than just a regular data array here. 

In [ ]:
ds_prism_chunk = rxr.open_rasterio(tif_filename, masked=True, chunks={"x": 2048, "y": 2048})
%timeit ds_prism_chunk.mean().compute()

In [ ]:
ds_prism = rxr.open_rasterio(tif_filename, masked=True)
%timeit ds_prism.mean()

Even slower with larger chunks of 2048 x 2048. 